In [1]:
contest_folder = 'practice'

In [14]:
import glob
import os

from worker import problem_name_to_problem_code

folders = glob.glob(f"./{contest_folder}/*/")  # only select folders

problem_names = [os.path.basename(os.path.normpath(folder)) for folder in folders]
problem_mapping = {problem_name_to_problem_code(problem_name): problem_name for problem_name in problem_names}

problem_names

['Line of Delivery (Part 1)',
 'Line of Delivery (Part 2)',
 'Line by Line',
 'Fall in Line',
 'Walk the Line']

In [17]:
# Get all files in execution_full_out
files = glob.glob(f"./execution_*/*/*")  # get all files, e.g., 'fall_in_line' and '000'

# Extract execution type, problem name, and file ID
file_info = [
    (
        os.path.basename(os.path.dirname(os.path.dirname(file))),
        os.path.basename(os.path.dirname(file)),
        os.path.splitext(os.path.basename(file))[0]
    ) for file in files
]

# file_info

In [16]:
import pandas as pd

# Create a DataFrame from file_info and files
df = pd.DataFrame(file_info, columns=['execution_type', 'problem', 'id'])
df['filepath'] = files

# Pivot the DataFrame to have execution types as separate columns
df_grouped = df.pivot(index=['problem', 'id'], columns='execution_type', values='filepath').reset_index()

# Rename columns to include '_filepath'
df_grouped.columns = ['problem', 'id'] + [f"{col}_filepath" for col in df_grouped.columns if col not in ['problem', 'id']]

# Function to read file content
def read_file(problem_code, filename):
    problem_name = problem_mapping.get(problem_code)
    if problem_name:
        filepath = os.path.join('practice', problem_name, filename)
        if os.path.exists(filepath):
            with open(filepath, 'r') as f:
                return f.read()
    return None

# Add 'statement', 'sample_in', 'sample_out' columns
df_grouped['statement'] = df_grouped['problem'].apply(lambda x: read_file(x, 'statement.txt'))
df_grouped['sample_in'] = df_grouped['problem'].apply(lambda x: read_file(x, 'sample_in.txt'))
df_grouped['sample_out'] = df_grouped['problem'].apply(lambda x: read_file(x, 'sample_out.txt'))

# Function to read file content from a given filepath
def read_file_content(filepath):
    if filepath and os.path.exists(filepath):
        with open(filepath, 'r') as f:
            return f.read()
    return None

# Replace 'execution_code_filepath' with the actual code content
df_grouped['execution_code'] = df_grouped['execution_code_filepath'].apply(read_file_content)

# Replace 'execution_sample_out_filepath' with the actual sample output content
df_grouped['execution_sample_out'] = df_grouped['execution_sample_out_filepath'].apply(read_file_content)

# Replace 'execution_full_out_filepath' with the actual full output content
df_grouped['execution_full_out'] = df_grouped['execution_full_out_filepath'].apply(read_file_content)

# Optionally, drop the original filepath columns if they are no longer needed
df_grouped = df_grouped.drop(columns=['execution_code_filepath', 'execution_full_out_filepath', 'execution_sample_out_filepath'])

# Display the updated DataFrame with file contents instead of filepaths
df_grouped

,problem,id,statement,sample_in,sample_out,execution_code,execution_sample_out,execution_full_out
0,fall_in_line,000,"As the queen of an ant colony, it’s your job t...",3\n4\n1 1\n2 2\n-3 -3\n4 4\n4\n1 1\n-1 1\n1 -1...,Case #1: 0\nCase #2: 2\nCase #3: 3\n,from collections import defaultdict\nimport sy...,Case #1: 0\nCase #2: 2\nCase #3: 3\n,
1,fall_in_line,001,"As the queen of an ant colony, it’s your job t...",3\n4\n1 1\n2 2\n-3 -3\n4 4\n4\n1 1\n-1 1\n1 -1...,Case #1: 0\nCase #2: 2\nCase #3: 3\n,from collections import defaultdict\nfrom math...,Case #1: 0\nCase #2: 2\nCase #3: 3\n,
2,line_by_line,000,You’ve found a solution to an implementation-h...,4\n2 50\n3 10\n13 37\n950 95\n,Case #1: 20.710678118654748\nCase #2: 11.54434...,"def calculate_required_increase(N, P):\n # ...",Case #1: 0.0\nCase #2: 0.0\nCase #3: 0.0\nCase...,Case #1: 0.0\nCase #2: 0.0\nCase #3: 0.0\nCase...
3,line_by_line,001,You’ve found a solution to an implementation-h...,4\n2 50\n3 10\n13 37\n950 95\n,Case #1: 20.710678118654748\nCase #2: 11.54434...,import sys\nimport math\n\ndef calculate_incre...,Case #1: 20.710678118654755\nCase #2: 0.0\nCas...,Case #1: 20.710678118654755\nCase #2: 0.0\nCas...
4,line_of_delivery_part_1,000,*This problem shares some similarities with pr...,4\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 3 4\nCase #3: 3 0\nCase...,def curling_solver(test_cases):\n results =...,Case #1: 1 5\nCase #2: 1 1\nCase #3: 1 7\nCase...,Case #1: 1 5\nCase #2: 1 1\nCase #3: 1 7\nCase...
5,line_of_delivery_part_1,001,*This problem shares some similarities with pr...,4\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 3 4\nCase #3: 3 0\nCase...,"def process_case(N, G, energies):\n stones ...",Error:\nTraceback (most recent call last):\n ...,Error:\nTraceback (most recent call last):\n ...
6,line_of_delivery_part_2,000,*This problem shares some similarities with pr...,5\n2 5\n7\n2\n3 6\n9\n5\n7\n4 8\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 2 2\nCase #3: 2 0\nCase...,def curling_simulation(test_cases):\n resul...,Case #1: 1 1\nCase #2: 2 3\nCase #3: 3 4\nCase...,
7,line_of_delivery_part_2,001,*This problem shares some similarities with pr...,5\n2 5\n7\n2\n3 6\n9\n5\n7\n4 8\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 2 2\nCase #3: 2 0\nCase...,def curling_game(test_cases):\n results = [...,Case #1: 2 4\nCase #2: 3 4\nCase #3: 4 5\nCase...,
8,walk_the_line,000,"There’s an old, narrow bridge that a group of ...",6\n4 17\n1\n2\n5\n10\n4 4\n1\n2\n5\n10\n2 22\n...,Case #1: YES\nCase #2: NO\nCase #3: YES\nCase ...,"def can_cross_bridge(travelers, K):\n N = l...",Case #1: NO\nCase #2: NO\nCase #3: YES\nCase #...,Case #1: NO\nCase #2: NO\nCase #3: YES\nCase #...
9,walk_the_line,001,"There’s an old, narrow bridge that a group of ...",6\n4 17\n1\n2\n5\n10\n4 4\n1\n2\n5\n10\n2 22\n...,Case #1: YES\nCase #2: NO\nCase #3: YES\nCase ...,"def can_cross(N, K, S):\n # Special case: I...",Case #1: YES\nCase #2: NO\nCase #3: YES\nCase ...,Case #1: YES\nCase #2: NO\nCase #3: YES\nCase ...
